# Reading the coronavirus from Sciensano with Python

**Author:** Pierre de Buyl  
**Date:** 4 march 2021  
**Licence:** [CC-BY](https://creativecommons.org/licenses/by/4.0/)

In this notebook, I present how to load the files published by Sciensano at
https://epistat.wiv-isp.be/covid/

The files are available in csv and present a non-unique index as the lines
correspond to a set of date/province (for hospitalization data) and
date/province/agegroup/sex for the cases.

I use [pandas](https://pandas.pydata.org/) for its read_csv, groupby and rolling mean features.
Also [matplotlib](https://matplotlib.org/) and [NumPy](https://numpy.org/).

See https://github.com/pdebuyl/coronavirus_notebooks for the notebook file. You can
execute the notebook online at https://mybinder.org/v2/gh/pdebuyl/coronavirus_notebooks/master

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os
import os.path

In [ ]:
# Give room for the tilted date labels
plt.rcParams['figure.subplot.bottom'] = 0.2
plt.rcParams['figure.subplot.hspace'] = 0.3
plt.rcParams['figure.max_open_warning'] = 100

def simplified_format(x, pos):
    """
    Simplified formatting for the y axis of logarithmic graphs
    """
    if x >1:                                      
        return '%1.0f' % x     
    else:                      
        return '%g' % x


In [ ]:
# Change the line below to the location of the CSV files
data_directory = 'data'

# Read hospitalisation and cases data
# File encoding must be set, the default from pandas is utf-8

df_HOSP =  pd.read_csv(os.path.join(data_directory, 'COVID19BE_HOSP.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE'])

df_CASES = pd.read_csv(os.path.join(data_directory, 'COVID19BE_CASES_AGESEX.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE', 'AGEGROUP', 'SEX'])

df_TESTS = pd.read_csv(os.path.join(data_directory, 'COVID19BE_tests.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE'])

df_MORT = pd.read_csv(os.path.join(data_directory, 'COVID19BE_MORT.csv'),
                       encoding='utf-8', index_col=['DATE', 'REGION', 'AGEGROUP', 'SEX'])


## Plotting the data

I consider the hospitalization data for all provinces summed, as it is
the main reporting done in the media and in the comparisons with epidemiologic
models.

Some data are daily values, such as the number of *new* hospitalizations. Others
are cumulative, i.e. the sum to date for every data point, such as cumulative cases.

In [ ]:
HOSP_bydate = df_HOSP.groupby('DATE').sum()

CASES_bydate = df_CASES.groupby('DATE').sum()

TESTS_bydate = df_TESTS.groupby('DATE').sum() #[:-1]


HOSP_time = np.asarray(HOSP_bydate.index, dtype=np.datetime64)
CASES_time = np.asarray(CASES_bydate.index, dtype=np.datetime64)
TESTS_time = np.asarray(TESTS_bydate.index, dtype=np.datetime64)

MORT_time = np.asarray(df_MORT.groupby('DATE').sum().index, dtype=np.datetime64)

In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN'], label='in hospital')

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN_ICU'], label='in ICU')

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.legend()
plt.yscale('log')
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");
ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(simplified_format))

plt.title("COVID19 Belgium - hospital occupancy\nData source: Sciensano - Figure: P. de Buyl")
plt.savefig('/tmp/COVID19BE_hospital.png')

plt.figure()

TESTS_rm = TESTS_bydate['TESTS_ALL'].rolling(7).mean()
CASES_rm = CASES_bydate['CASES'].rolling(7).mean()

plt.plot(TESTS_time, CASES_rm/TESTS_rm)

plt.yscale('log')
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");
ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(simplified_format))

plt.title("COVID19 Belgium - Positive rate of tests (7 days mean)\nData source: Sciensano - Figure: P. de Buyl")
plt.savefig('/tmp/COVID19BE_positivity.png')

plt.figure()

deaths_all = df_MORT.groupby(level=[0,1,2]).sum().groupby('DATE').sum()

plt.plot(MORT_time, deaths_all.rolling(7).mean())
plt.plot(MORT_time, np.cumsum(deaths_all['DEATHS']))

plt.yscale('log')
plt.grid()

ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");
ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(simplified_format))

plt.title("COVID19 Belgium - Daily deaths (7 days mean and cumulative count)\nData source: Sciensano - Figure: P. de Buyl")
plt.savefig('/tmp/COVID19BE_deaths.png')


In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN']/HOSP_bydate['TOTAL_IN_ICU'])

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.title("COVID19 Belgium - ratio of total / ICU beds")

In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN'], label='in hospital')

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN_ICU'], label='in ICU')

plt.plot(CASES_time, CASES_bydate['CASES'], label='cases')
plt.plot(CASES_time, np.cumsum(CASES_bydate['CASES']), label='cumul cases')

plt.plot(TESTS_time, TESTS_bydate['TESTS_ALL'], label='tests')

plt.axhline(2200)

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.legend()
plt.yscale('log')
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.title('Daily indicators and cumulative cases')

## Plotting the cases and hospitalization numbers per province

In [ ]:
cases_prov = df_CASES.groupby(level=[0,1]).sum().groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(cases_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = cases_prov.get_group(k)
    g = g.reset_index(level=1, drop=True)
    g.rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Cases per province, rolling mean')

In [ ]:
hosp_prov = df_HOSP.groupby('PROVINCE') #.reset_index(level=1, drop=True)

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(hosp_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = hosp_prov.get_group(k).reset_index(level=1, drop=True)
    g['TOTAL_IN'].rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    g['TOTAL_IN_ICU'].rolling(7).mean().plot(ax=ax, legend=i==1, rot=45, ls='--')
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Hospitalization per province, rolling mean')

In [ ]:
tests_prov = df_TESTS.groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(tests_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = tests_prov.get_group(k).reset_index(level=1, drop=True)
    g.rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Tests per province, rolling mean')

In [ ]:
tests_prov = df_TESTS.groupby('PROVINCE')
cases_prov = df_CASES.groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(tests_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    tests_group = tests_prov.get_group(k).reset_index(level=1, drop=True).groupby('DATE').sum().rolling(7).mean()[1:-1]
    cases_group = cases_prov.get_group(k).groupby('DATE').sum().rolling(7).mean()
    positivity = cases_group['CASES']/tests_group['TESTS_ALL']
    positivity.plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Positivity per province, rolling mean')

In [ ]:
cases_agegroup = df_CASES.groupby(level=[0,2]).sum().groupby('AGEGROUP')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(cases_agegroup.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = cases_agegroup.get_group(k)
    g = g.reset_index(level=1, drop=True)
    g.plot(ax=ax)
    g.rolling(7).mean().plot(rot=45, label='rolling mean', ax=ax)
    plt.yscale('log')
    plt.title(f'Age range {k}')
    plt.grid()
    legend = ax.legend()
    legend.remove()
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.setp(axes[-1][-1].get_xticklabels(), rotation=30, ha="right");

plt.suptitle("COVID19 Belgium - Cases per age group\nData source: Sciensano - Figure: P. de Buyl")


In [ ]:
new = HOSP_bydate['NEW_IN']

rolling_length = 7
daily_growth_rate = (
(HOSP_bydate['NEW_IN'].rolling(rolling_length).mean() - HOSP_bydate['NEW_OUT'].rolling(rolling_length).mean())
/ HOSP_bydate['TOTAL_IN'].rolling(rolling_length).mean()
)

text_offsets = [
    [(-100, -10), (40, 25)],
    [(-40, -100), (-60, -40), (20, 35)]
]
    
minmax = [(30, 150), (150, len(HOSP_time)-1)]
annotations = [
    [31, 150, ],
    [151, 261, len(HOSP_time)-1]
]

for ((min, max), offsets, indices) in zip(minmax, text_offsets, annotations):
    fig = plt.figure()

    days_since_march_15 = np.array(HOSP_time - HOSP_time[0], dtype=float)

    # Select given time period
    mask = (days_since_march_15 > min) * (days_since_march_15 <= max)

    plt.plot(new[mask], daily_growth_rate[mask]*100, marker='.')
    plt.plot(new[mask][::7], daily_growth_rate[mask][::7]*100, marker='o', ls='')

    plt.xlabel('New hospitalizations per day')
    plt.ylabel(f'Growth rate in percent\n[(in - out)/total * 100] ({rolling_length} days av.)')

    for o, a in zip(offsets, indices):
        plt.annotate("{}".format(HOSP_time[a]),
                     (new[a], daily_growth_rate[a]*100),
                     arrowprops=dict(arrowstyle="simple"),
                     xytext=o, textcoords="offset points"
                    )

    plt.axhline(0, color='gray')
    plt.grid()
    plt.xscale('log')
    plt.gca().xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(simplified_format))
    
    title_text = """COVID19 Belgium - Hospitalization barometer {} - {}
Data source: Sciensano - Figure: P. de Buyl""".format(HOSP_time[min], HOSP_time[max])

    plt.title(title_text)
    plt.savefig('/tmp/COVID19BE_barometer_{}_{}.png'.format(HOSP_time[min], HOSP_time[max]))
    plt.legend(['one point per day', 'one point per week'])


